In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_columns', None)

https://www.kaggle.com/datasets/whenamancodes/student-performance

In [2]:
# Reads the data
math = pd.read_csv('Maths.csv', encoding='ISO8859-1', sep=';')
port = pd.read_csv('Portuguese.csv', encoding='ISO8859-1', sep=';')

# Adds an 'id' column with incremental values to the largest dataframe (port) and with 'x' to the other dataframe
id_column = range(1, len(port) + 1)
x_column = ['x' for _ in range(len(math))]
port.insert(0, 'id', id_column)
math.insert(0, 'id', x_column)

port.head(5)
math.head(5)

,id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,2,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,3,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,4,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,5,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


,id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,x,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,x,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,x,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,x,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,x,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [3]:
# Gets only the columns used to identify the same student in both dataframes, dropping 'id', their grades, 'failures' and 'paid'
identifiers = math.columns[1:-4]
identifiers = np.delete(identifiers, np.where(np.isin(identifiers, ['failures', 'paid']))[0])

# Creates two dataframes with the identifiers columns
math_id = math[identifiers]
port_id = port[identifiers]

# Iterates over 'math' to check if the same student is present in 'port'
for i, row in math_id.iterrows():

    match = port_id[port_id.eq(row).all(axis=1)]

    if not match.empty:
        # If there is a match, assigns the corresponding 'id' from 'port' to 'math'
        math.at[i, 'id'] = port.at[match.index[0], 'id']
    
math['id'].value_counts()

x      25
1       2
640     2
290     1
288     1
       ..
122     1
121     1
120     1
119     1
649     1
Name: id, Length: 371, dtype: int64

In [4]:
# There are 25 students who apparently took only math classes, not portuguese, so they will be removed. Duplicates will also be removed
math = math[math['id'] != 'x'].drop_duplicates(subset=['id'], ignore_index=True)
math

,id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,2,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,3,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,4,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,5,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,645,MS,F,19,R,GT3,T,2,3,services,other,course,mother,1,3,1,no,no,no,yes,no,yes,yes,no,5,4,2,1,2,5,0,7,5,0
366,646,MS,F,18,U,LE3,T,3,1,teacher,services,course,mother,1,2,0,no,yes,yes,no,yes,yes,yes,no,4,3,4,1,1,1,0,7,9,8
367,647,MS,F,18,U,GT3,T,1,1,other,other,course,mother,2,2,1,no,no,no,yes,yes,yes,no,no,1,1,1,1,1,5,0,6,5,0
368,648,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,3,14,16,16


In [10]:
# Merges the two dataframes based on 'id'
df = pd.merge(port, math, on='id', how='right', suffixes=('', '_math'))

# Removes duplicated columns, keeping only the different ones
identifiers_math = [name + '_math' for name in identifiers]
df = df.drop(identifiers_math, axis=1)

# Adds the suffix '_port' on columns from the 'port' dataframe
port_exclusive = ['failures', 'paid'] + list(port.columns[-4:])
identifiers_port = [name + '_port' for name in port_exclusive]
column_mapping = {port_exclusive[i]: identifiers_port[i] for i in range(len(port_exclusive))}
df.rename(columns=column_mapping, inplace=True)

# Displays the merged dataframe
df

,id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures_port,schoolsup,famsup,paid_port,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences_port,G1_port,G2_port,G3_port,failures_math,paid_math,absences_math,G1_math,G2_math,G3_math
0,1,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11,0,no,6,5,6,6
1,2,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11,0,no,4,5,5,6
2,3,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12,3,yes,10,7,8,10
3,4,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14,0,yes,2,15,14,15
4,5,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13,0,yes,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,645,MS,F,19,R,GT3,T,2,3,services,other,course,mother,1,3,1,no,no,no,yes,no,yes,yes,no,5,4,2,1,2,5,4,10,11,10,1,no,0,7,5,0
366,646,MS,F,18,U,LE3,T,3,1,teacher,services,course,mother,1,2,0,no,yes,no,no,yes,yes,yes,no,4,3,4,1,1,1,4,15,15,16,0,yes,0,7,9,8
367,647,MS,F,18,U,GT3,T,1,1,other,other,course,mother,2,2,0,no,no,no,yes,yes,yes,no,no,1,1,1,1,1,5,6,11,12,9,1,no,0,6,5,0
368,648,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,6,10,10,10,0,no,3,14,16,16


In [11]:
# Calculates the average of the 3 results for _port and _math
df['GM_port'] = df[['G1_port', 'G2_port', 'G3_port']].mean(axis=1)
df['GM_math'] = df[['G1_math', 'G2_math', 'G3_math']].mean(axis=1)
df

,id,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures_port,schoolsup,famsup,paid_port,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences_port,G1_port,G2_port,G3_port,failures_math,paid_math,absences_math,G1_math,G2_math,G3_math,GM_port,GM_math
0,1,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11,0,no,6,5,6,6,7.333333,5.666667
1,2,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11,0,no,4,5,5,6,10.333333,5.333333
2,3,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12,3,yes,10,7,8,10,12.333333,8.333333
3,4,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14,0,yes,2,15,14,15,14.000000,14.666667
4,5,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13,0,yes,4,6,10,10,12.333333,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,645,MS,F,19,R,GT3,T,2,3,services,other,course,mother,1,3,1,no,no,no,yes,no,yes,yes,no,5,4,2,1,2,5,4,10,11,10,1,no,0,7,5,0,10.333333,4.000000
366,646,MS,F,18,U,LE3,T,3,1,teacher,services,course,mother,1,2,0,no,yes,no,no,yes,yes,yes,no,4,3,4,1,1,1,4,15,15,16,0,yes,0,7,9,8,15.333333,8.000000
367,647,MS,F,18,U,GT3,T,1,1,other,other,course,mother,2,2,0,no,no,no,yes,yes,yes,no,no,1,1,1,1,1,5,6,11,12,9,1,no,0,6,5,0,10.666667,3.666667
368,648,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,6,10,10,10,0,no,3,14,16,16,10.000000,15.333333
